In [1]:
import torch
import gym
from gym import envs
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.autograd import Variable

In [2]:
lenobs = 100800
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.l1 = nn.Linear(lenobs,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,2)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.relu(self.l2(y))
        actor = F.softmax(self.actor_lin1(y),dim=0)
#         actor = F.log_softmax(self.actor_lin1(y),dim=0)
        c = F.relu(self.l3(y.detach()))
        critic = torch.tanh(self.critic_lin1(c))
        return actor, critic

In [5]:
env = gym.make('PongNoFrameskip-v0')
moveMapping = {
    0:2,
    1:3
}

model = ActorCritic()
optimizer = optim.Adam(lr=1e-4,params=model.parameters())

'''
loss = Variable(loss, requires_grad = True)
actor_loss = Variable(actor_loss, requires_grad = True)
critic_loss = Variable(critic_loss, requires_grad = True)
'''

for i_episode in range(100):
#     reward = 0.0
    values = []
    rewards = []
    logprobs = []
    observation = env.reset()
    print('---------------')
    done = False
    N = 0
    while N<10 and done == False:
        N+=1
#         print(t)
        pobservation = torch.from_numpy(observation)
        flattened_pobservation = pobservation.view(-1).float()
        policy, value = model(flattened_pobservation)
#         print(policy)
        values.append(value)
        action = np.random.choice(np.array([0,1]), p = policy.view(2,).data.numpy())
        logprobs.append(policy.view(-1)[action])
#         print('Action: {}'.format('right' if action==2 else 'left'))
        observation, reward, done, log = env.step(moveMapping[action])
        rewards.append(reward)
#         print('{}.Reward: {}'.format(t,reward))
#         print('---')
        if done:
            print(done)
            print('Episode:{} State:{} Reward:{}'.format(i_episode,t,reward))
            print("Episode finished after {} timesteps".format(N+1))    
            break
        else:
            G = value
        
      
    # Reversing because earlier actions have greater value
    torch_values = torch.tensor(values, requires_grad = True).view(-1).flip(0)
    torch_rewards = torch.tensor(rewards, requires_grad = True).flip(0)
    torch_logprobs = torch.tensor(logprobs, requires_grad = True).flip(0)
    
    
    returns = []
    gamma = 0.95
    clc = 0.1
    ret = torch.tensor([G])
    for r in torch_rewards:
        ret = r + gamma*ret
        returns.append(ret)
    returns = torch.tensor(returns, requires_grad = True)
    returns = F.normalize(returns,dim=0)
    actor_loss = -1*torch_logprobs * (returns - torch_values)
    critic_loss = torch.pow(torch_values - returns,2)
    loss = actor_loss.sum() + clc*critic_loss.sum()
    print('Loss: {}'.format(loss))
#     print('Starting Backpropagation')
    loss.backward()
    optimizer.step()
#     print('Completed Backpropagation')


---------------
Loss: -0.9341222047805786
---------------
Loss: -0.9183687567710876
---------------
Loss: -0.9199767112731934
---------------
Loss: -0.9275506138801575
---------------
Loss: -0.94156813621521
---------------
Loss: -0.920374870300293
---------------
Loss: -0.9271572828292847
---------------
Loss: -0.932810366153717
---------------
Loss: -0.9277443885803223
---------------
Loss: -0.9345334768295288
---------------
Loss: -0.9212775230407715
---------------
Loss: -0.9217564463615417
---------------
Loss: -0.9332188367843628
---------------
Loss: -0.926699161529541
---------------
Loss: -0.9226280450820923
---------------
Loss: -0.9416612386703491
---------------
Loss: -0.9247136116027832
---------------
Loss: -0.9294534921646118
---------------
Loss: -0.9369943141937256
---------------
Loss: -0.9251651763916016
---------------
Loss: -0.9361915588378906
---------------
Loss: -0.9354326128959656
---------------
Loss: -0.9232856035232544
---------------
Loss: -0.92580831050872